# Problems

## Preamble

In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.integrate import quad
from scipy.interpolate import InterpolatedUnivariateSpline as interpolate
from scipy.optimize import root
from scipy.misc import derivative
import scipy.linalg as lg

import sympy as sp
import fractions as fra
import itertools as itr

from tabulate import tabulate

from ipywidgets import interact

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib as mpl
import matplotlib.pyplot as plt

import matplotlib.ticker as ticker
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
%run -i "rcParameters.py"
#from mpl_toolkits.mplot3d import Axes3D

plt.rcParams["font.family"]='Times New Roman'
plt.rcParams["mathtext.fontset"]='stix';

In [ ]:
# set fontsize in markdown cells
from IPython.core.display import display, HTML, Latex
display(HTML("<style>.rendered_html { font-size: 16px; }</style>"))

## Utility functions

In [ ]:
def NestList(f,x0,n0):
    '''
    generate nested list by applying f n=0,...,n0 times on x0
    input:  f(x,_), x0, n0=integer
    output: [x0,f(x0),f(f(x0)),...,f(...(f(x0)))]
    '''
    return [a for a in itr.accumulate(itr.repeat(x0,n0),f)]

def Nest(f,x,n):
    '''Applies function f n-times to x'''
    if n==0:
        return x
    return Nest(f,f(x),n-1)

In [ ]:
# define Logistic Map (LM) and LM series
f_LM = lambda x,r: r*x*(1-x)
def logistic_series(x0,r,n):
    f = lambda x,_: f_LM(x,r)
    return NestList(f,x0,n)

In [ ]:
# plot cobweb x_(n+1)=f(x_n)
def cobweb_f(f,x0,n0,ax,ls='-',col='r',lw=1):
    ax = ax or plt.gca
    f_ = lambda x,_: f(x)
    dat = NestList(f_,x0,n0)
    x = list(itr.chain(*zip(dat,dat)))
    ax.plot(x[:-1],x[1:],c=col,ls=ls,lw=lw)
    return ax

# plot cobweb {x_0,...,x_n}
def cobweb_x(xdat,ax,ls='-',col='r',lw=1,nS=1):
    ax = ax or plt.gca
    x = list(itr.chain(*zip(xdat,xdat)))
    ax.plot(x[nS:-1],x[nS+1:],c=col,ls=ls,lw=lw)
    return ax

## 10.3.7

Consider the ___decimal shift map___ on the unit interval given by 

$x_{n+1}=10\,x_n\;$(mod 1)

As usual, "$z (mod 1)$" means the noninteger part of $z$.

#### (a) draw the graph of the map

In [ ]:
f = lambda x: 10*x % 1
fp= lambda x: np.diff(f(x))

In [ ]:
xx = np.linspace(0,1,600)
# remove discontinuities
yy = f(xx)
dp = np.where(np.diff(f(xx)) < 0)[0]+1
yy1 = np.insert(yy,dp,np.nan)
xx1=np.copy(xx)
xx1=np.insert(xx,dp,np.nan)

# plot f(x) w/o discontinuities
plt.plot(xx,xx,'k--',lw=1)
plt.plot(xx1,yy1,'b-',lw=2)
# plot the fixed points x*=f(x*) = {0,1/9,2/9,...,8/9,1=0.999...}
for i in np.arange(0,10,1):
    plt.plot(i/9,i/9,'o',ms=8,mfc='w',mec='r',lw=1)
plt.grid()
plt.xlabel('$x$')
plt.ylabel('$f(x)$')
# plt.xlim(0,1)
# plt.ylim(0,1)
# plt.yticks(np.arange(0.2,1.2,0.2))
plt.show()

In [ ]:
# cobweb for two nearby ICs
def f_series(x0,n):
    f_ = lambda x,_: f(x)
    return NestList(f_,x0,n)

x1=f(np.pi)
x2=x1*1.001

fig, ax=plt.subplots()
cobweb_x(f_series(x1,100)[:],ax,lw=1,col='r')
cobweb_x(f_series(x2,100)[:],ax,lw=1,col='b')

ax.plot(x1,f(x1),'o',ms=8,mfc='r',mec='b',mew=2)
ax.plot(xx,xx,'k--')
ax.plot(xx1,f(xx1),'k-',lw=2,alpha=0.4)

ax.set_aspect('equal')
ax.grid()
ax.set_xlabel('$x_n$')
ax.set_ylabel('$x_{n+1}$')
# ax.set_xticks(np.arange(0,1.2,0.2))
plt.show()

## P10.4.3

The map $x_{n+1}=1-rx^2$ has a superstable 3-cycle at a certain value of $r$. Find a cubic equation for $r$.

#### Orbit diagram

In [ ]:
f2 = lambda x,r: 1-r*x**2
f2x = lambda x: f2(x,r)
# define f^n(x,r)
f2n = lambda x,n: Nest(f2x,x,n)

def f2_series(x0,r,n):
    f = lambda x,_: f2(x,r)
    return NestList(f,x0,n)

In [ ]:
x,y,r = sp.symbols('x,y,r',real=True)

# equation: d[f^3(x,r)]/dx=0
eq3p = f2n(x,3).diff(x)
display(eq3p)
[display(u.simplify()) for u in sp.solve(eq3p,x)];

In [ ]:
# find cubic eq for r
eq_r = f2n(0,3).expand()
display(eq_r)
# find value of r for superstable 3-cylcle
r0 = sp.solve(eq_r,r)[2]
display(r0)
r0.n(6)

In [ ]:
x_s = [u.n(6) for u in sp.solve(eq3p.subs(r,r0.n(6)),x)]
x_s

In [ ]:
[f2n(u,3).subs(r,r0.n(6)) for u in x_s]

In [ ]:
r_1 = 1; r_2 = 2
s1=301; s2=450
x_0 = 1; x_1=-1.2; x_2=1.2
col='r'

fig_OD, ax=plt.subplots(figsize=(8,6))

for u in np.linspace(r_1,r_2,200):
    yy=f2_series(x_0,u,s2)[s1:]
    yy2=f2_series(-x_0,u,s2)[s1:]
    xx=np.ones(len(yy))*u
    ax.plot(xx,yy,'o',ms=0.5,mfc=col,mec=col)
    ax.plot(xx,yy2,'o',ms=0.5,mfc=col,mec=col)

ax.axvline(r0.n(6),x_1,x_2,color='b',lw=1,label="$r=1.75488$")
ax.grid()
ax.legend(fontsize=20,framealpha=0.7)
ax.set_ylim(-1.2,1.2)
ax.set_xlabel('$r$')
ax.set_ylabel('$x$')
plt.show()

In [ ]:
# cobweb for r0=1.75488
x1=0.5
xx = np.linspace(-1,1,50)

fig, ax=plt.subplots()
cobweb_x(f2_series(x1,r0.n(6),100)[80:],ax,lw=2)

ax.plot(xx,xx,'k--')
ax.plot(xx,f2(xx,r0),'k-',lw=2)

ax.set_aspect('equal')
ax.grid()
ax.set_xlabel('$x_n$')
ax.set_ylabel('$x_{n+1}$')
# ax.set_xticks(np.arange(0,1.2,0.2))
plt.show()

In [ ]:
eq_ = 1-x*(1-x)**2
[u.n(6) for u in sp.solve(eq_,x)]